<a href="https://colab.research.google.com/github/Vakhranev/MDB/blob/main/%D0%93%D0%B5%D0%B9%D0%BC%D1%81-%D0%A5%D0%B0%D1%83%D1%8D%D0%BB%D0%BB%20%D0%B4%D0%BB%D1%8F%20%D0%B4%D0%BB%D0%B8%D0%BD%D1%8B%20%D1%82%D0%BE%D0%BA%D0%B5%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from scipy.stats import f_oneway
import os
import glob
import numpy as np
from scipy.stats import f_oneway

cefr_levels = ["A1", "A2", "B1", "B2"]
# Здесь следует добавить обработку файлов и расчет средних значений токенов
# И после этого, вы можете использовать код ANOVA, подобный предыдущему примеру.

# Список для хранения длин токенов всех файлов
all_token_lengths = []

# Словарь для хранения средних значений длин токенов в каждом уровне CEFR
avg_lengths_in_level = {}

# Обработка каждого уровня CEFR
for cefr_level in cefr_levels:
    folder_path = f"/content/Students_texts/{cefr_level}"
    file_list = glob.glob(os.path.join(folder_path, "*.txt"))

    # Список для хранения средних значений длин токенов в текущем уровне CEFR
    lengths_in_level = []

    # Обработка каждого файла в текущем уровне CEFR
    for file_path in file_list:
        with open(file_path, 'r') as file:
            text = file.read()
            tokens = text.split()
            token_lengths = [len(token) for token in tokens]
            avg_token_length = np.mean(token_lengths)
            lengths_in_level.append(avg_token_length)
            all_token_lengths.extend(token_lengths)

    # Среднее значение для текущего уровня CEFR
    avg_for_cefr = np.mean(lengths_in_level)
    avg_lengths_in_level[cefr_level] = lengths_in_level

# ANOVA
f_statistic, p_value = f_oneway(
    avg_lengths_in_level['A1'],
    avg_lengths_in_level['A2'],
    avg_lengths_in_level['B1'],
    avg_lengths_in_level['B2']
)

print(f"F-статистика: {f_statistic}")
print(f"p-value: {p_value}")

F-статистика: 57.30753093011176
p-value: 2.0828233774138275e-34


In [3]:
from scipy.stats import bartlett

group_obs = {level: len(avg_lengths_in_level[level]) for level in cefr_levels}
group_means = {level: np.mean(avg_lengths_in_level[level]) for level in cefr_levels}
group_variance = {level: np.var(avg_lengths_in_level[level]) for level in cefr_levels}

# Degrees of freedom for each group
k = len(cefr_levels)

# Calculate pooled variance
pooled_variance = sum([(group_obs[level] - 1) * group_variance[level] for level in cefr_levels]) / (sum(group_obs.values()) - k)

# Calculate test statistic
games_howell_statistic = []
for i in range(k - 1):
    for j in range(i + 1, k):
        t_val = np.abs(group_means[cefr_levels[i]] - group_means[cefr_levels[j]]) / np.sqrt(
            pooled_variance * (1 / group_obs[cefr_levels[i]] + 1 / group_obs[cefr_levels[j]])
        )
        df_num = (pooled_variance ** 2) / (
            (group_variance[cefr_levels[i]] ** 2 / (group_obs[cefr_levels[i]] ** 2 * (group_obs[cefr_levels[i]] - 1))) +
            (group_variance[cefr_levels[j]] ** 2 / (group_obs[cefr_levels[j]] ** 2 * (group_obs[cefr_levels[j]] - 1)))
        )
        df_denom = (
            (group_variance[cefr_levels[i]] ** 2 / (group_obs[cefr_levels[i]] ** 2 * (group_obs[cefr_levels[i]] - 1))) ** 2 / (group_obs[cefr_levels[i]] - 1) +
            (group_variance[cefr_levels[j]] ** 2 / (group_obs[cefr_levels[j]] ** 2 * (group_obs[cefr_levels[j]] - 1))) ** 2 / (group_obs[cefr_levels[j]] - 1)
        )
        df = df_num / df_denom

        games_howell_statistic.append((cefr_levels[i], cefr_levels[j], t_val, df))

# Printing results
for stat in games_howell_statistic:
    print(f"Comparison: {stat[0]} vs {stat[1]}")
    print(f"t-value: {stat[2]}")
    print(f"Degrees of freedom: {stat[3]}")

Comparison: A1 vs A2
t-value: 1.7785874042623104
Degrees of freedom: 4289494830447953.0
Comparison: A1 vs B1
t-value: 2.628812709513902
Degrees of freedom: 4288804776973837.5
Comparison: A1 vs B2
t-value: 6.318112696493338
Degrees of freedom: 4210314532547371.0
Comparison: A2 vs B1
t-value: 8.915254260425465
Degrees of freedom: 4.4781735174074354e+27
Comparison: A2 vs B2
t-value: 12.03860137182147
Degrees of freedom: 1.3408701775113266e+21
Comparison: B1 vs B2
t-value: 6.206896625096317
Degrees of freedom: 1.3293849424075075e+21


In [4]:
import pandas as pd

result_dict = {
    'Comparison': [f"{stat[0]} vs {stat[1]}" for stat in games_howell_statistic],
    't-value': [stat[2] for stat in games_howell_statistic],
    'Degrees of freedom': [stat[3] for stat in games_howell_statistic]
}

result_df = pd.DataFrame(result_dict)
print(result_df)

  Comparison    t-value  Degrees of freedom
0   A1 vs A2   1.778587        4.289495e+15
1   A1 vs B1   2.628813        4.288805e+15
2   A1 vs B2   6.318113        4.210315e+15
3   A2 vs B1   8.915254        4.478174e+27
4   A2 vs B2  12.038601        1.340870e+21
5   B1 vs B2   6.206897        1.329385e+21
